https://www.kaggle.com/code/olyatsimboy/towards-tf-idf-in-logs-features

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/'My Drive'/'kaggle'/'LinkingWritingProcess'

/content/drive/My Drive/kaggle/LinkingWritingProcess


In [3]:
!pip install optuna

## (1) Imports

In [4]:
import gc
import os
import itertools
import pickle
import re
import time

import warnings
warnings.filterwarnings('ignore')

from random import choice, choices
from functools import reduce
from tqdm import tqdm
from itertools import cycle

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
%matplotlib inline

from collections import Counter
from functools import reduce
from tqdm import tqdm
from itertools import cycle
from scipy import stats
from scipy.stats import skew, kurtosis
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import ensemble
from sklearn import decomposition
from sklearn import tree

import lightgbm as lgb
import xgboost as xgb

import optuna

pd.set_option("display.max_columns", None)

plt.style.use("ggplot")
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

In [5]:
tqdm.pandas()

sns.set_style('whitegrid')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.simplefilter('ignore')

import random
random.seed(42)

## (2) Load data

In [6]:
#INPUT_DIR = '/kaggle/input/linking-writing-processes-to-writing-quality'
INPUT_DIR = './data'

train_logs = pd.read_csv(f'{INPUT_DIR}/train_logs.csv')
train_scores = pd.read_csv(f'{INPUT_DIR}/train_scores.csv')
test_logs = pd.read_csv(f'{INPUT_DIR}/test_logs.csv')

submission = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')

In [7]:
train_logs.shape, train_scores.shape, test_logs.shape, submission.shape

((8405898, 11), (2471, 2), (6, 11), (3, 2))

In [8]:
train_logs.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [9]:
train_scores.head()

,id,score
0,001519c8,3.5
1,0022f953,3.5
2,0042269b,6.0
3,0059420b,2.0
4,0075873a,4.0


In [10]:
train_scores.describe()

,score
count,2471.000000
mean,3.711251
std,1.024937
min,0.500000
25%,3.000000
50%,4.000000
75%,4.500000
max,6.000000


## (3) EDA

In [11]:
#plt.figure(figsize=(15, 5))
#train_scores['score'].hist()
#plt.show()

In [12]:
#train_scores['score'].value_counts()

In [13]:
#event_stats = train_logs.groupby('id')['event_id'].count()

#fig, ax = plt.subplots(1, 2, figsize=(12, 5))
#ax[0].set_title('Distribution of events')
#ax[0].set_xlabel('Number of events in an essay')
#sns.histplot(event_stats, bins=100, ax=ax[0])
#ax[1].set_title('Boxplot of events')
#sns.boxplot(event_stats, ax=ax[1])
#plt.show()

In [14]:
#event_stats.describe()

In [15]:
#stats = train_logs.groupby('id')['event_id'].max().reset_index()
#stats_score = stats.merge(train_scores, on='id')

#catplot = sns.catplot(data=stats_score, x='score', y='event_id', kind='bar', height=5, aspect=8/5)
#plt.xlabel('Score', fontsize=12)
#plt.ylabel('Number of events', fontsize=12)
#plt.show()

In [16]:
tmp = train_logs['up_time'] - train_logs['down_time']
results = (tmp == train_logs['action_time'])
results.value_counts()

True    8405898
dtype: int64

In [17]:
#train_logs_scores_df = train_logs.merge(train_scores, on='id', how='left')
#train_logs_scores_df.head()

In [18]:
#train_logs_scores_df.columns

In [19]:
#train_logs_agg_df = train_logs_scores_df.groupby('id')[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count', 'score']].mean().reset_index()
#train_logs_agg_df

In [20]:
#def plot_dist_box(data, target):
#  color = choice(color_pal)

  # Create a figure with two subplot
#  fig, axes = plt.subplots(1, 4, figsize=(15, 4))

  # Plot the distribution plot on the first subplot
#  sns.histplot(data, ax=axes[0], color=color)
#  axes[0].set_title('Distribution Plot')

  # Plot the box plot on the second subplot
#  sns.boxplot(data, ax=axes[1], color=color)
#  axes[1].set_title('Box Plot')

  # Plot the box plot on the second subplot
#  sns.ecdfplot(data, ax=axes[2], color=color)
#  axes[2].set_title('CDF Plot')

  # Plot the box plot on the second subplot
#  sns.scatterplot(x=data, y=target, ax=axes[3], color=color)
#  axes[3].set_title('Scatter Plot')

  # Adjust the spacing between subplots
#  plt.tight_layout()
#  plt.show()

In [21]:
#num_cols = ['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']
#for col in num_cols:
#  plot_dist_box(train_logs_agg_df[col], train_logs_agg_df['score'])

In [22]:
#train_logs['activity'].value_counts()

In [23]:
#train_logs['down_event'].value_counts()

In [24]:
#train_logs['up_event'].value_counts()[:100]

In [25]:
#train_logs['text_change'].value_counts()[:100]

## (4) Feature Engineering

In [26]:
from collections import defaultdict

class Preprocessor:

    def __init__(self, seed):
        self.seed = seed

        self.activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
        self.events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',',
              'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
        self.text_changes = ['q', ' ', 'NoChange', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']
        self.punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                        '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']
        self.gaps = [1, 2, 3, 5, 10, 20, 50, 100]

        self.idf = defaultdict(float)
#         self.gaps = [1, 2]

    def activity_counts(self, df):
        tmp_df = df.groupby('id').agg({'activity': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['activity'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.activities:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'activity_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf

        return ret


    def event_counts(self, df, colname):
        tmp_df = df.groupby('id').agg({colname: list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df[colname].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.events:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'{colname}_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf

        return ret


    def text_change_counts(self, df):
        tmp_df = df.groupby('id').agg({'text_change': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['text_change'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.text_changes:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'text_change_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf

        return ret

    def match_punctuations(self, df):
        tmp_df = df.groupby('id').agg({'down_event': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['down_event'].values):
            cnt = 0
            items = list(Counter(li).items())
            for item in items:
                k, v = item[0], item[1]
                if k in self.punctuations:
                    cnt += v
            ret.append(cnt)
        ret = pd.DataFrame({'punct_cnt': ret})
        return ret


    def get_input_words(self, df):
        tmp_df = df[(~df['text_change'].str.contains('=>'))&(df['text_change'] != 'NoChange')].reset_index(drop=True)
        tmp_df = tmp_df.groupby('id').agg({'text_change': list}).reset_index()
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: ''.join(x))
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
        tmp_df['input_word_count'] = tmp_df['text_change'].apply(len)
        tmp_df['input_word_length_mean'] = tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_max'] = tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_std'] = tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df.drop(['text_change'], axis=1, inplace=True)
        return tmp_df

    def make_feats(self, df):

        print("Starting to engineer features")

        # initialize features dataframe
        feats = pd.DataFrame({'id': df['id'].unique().tolist()})

        # get shifted features
        # time shift
        print("Engineering time data")
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap)
            df[f'action_time_gap{gap}'] = df['down_time'] - df[f'up_time_shift{gap}']
        df.drop(columns=[f'up_time_shift{gap}' for gap in self.gaps], inplace=True)

        # cursor position shift
        print("Engineering cursor position data")
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f'cursor_position_shift{gap}'] = df.groupby('id')['cursor_position'].shift(gap)
            df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[f'cursor_position_shift{gap}']
            df[f'cursor_position_abs_change{gap}'] = np.abs(df[f'cursor_position_change{gap}'])
        df.drop(columns=[f'cursor_position_shift{gap}' for gap in self.gaps], inplace=True)

        # word count shift
        print("Engineering word count data")
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f'word_count_shift{gap}'] = df.groupby('id')['word_count'].shift(gap)
            df[f'word_count_change{gap}'] = df['word_count'] - df[f'word_count_shift{gap}']
            df[f'word_count_abs_change{gap}'] = np.abs(df[f'word_count_change{gap}'])
        df.drop(columns=[f'word_count_shift{gap}' for gap in self.gaps], inplace=True)

        # get aggregate statistical features
        print("Engineering statistical summaries for features")
        # [(feature name, [ stat summaries to add ])]
        feats_stat = [
            ('event_id', ['max']),
            ('up_time', ['max']),
            ('action_time', ['sum', 'max', 'mean', 'std']),
            ('activity', ['nunique']),
            ('down_event', ['nunique']),
            ('up_event', ['nunique']),
            ('text_change', ['nunique']),
            ('cursor_position', ['nunique', 'max', 'mean']),
            ('word_count', ['nunique', 'max', 'mean'])]
        for gap in self.gaps:
            feats_stat.extend([
                (f'action_time_gap{gap}', ['max', 'min', 'mean', 'std', 'sum', skew, kurtosis]),
                (f'cursor_position_change{gap}', ['max', 'mean', 'std', 'sum', skew, kurtosis]),
                (f'word_count_change{gap}', ['max', 'mean', 'std', 'sum', skew, kurtosis])
            ])

        pbar = tqdm(feats_stat)
        for item in pbar:
            colname, methods = item[0], item[1]
            for method in methods:
                pbar.set_postfix()
                if isinstance(method, str):
                    method_name = method
                else:
                    method_name = method.__name__

                pbar.set_postfix(column=colname, method=method_name)
                tmp_df = df.groupby(['id']).agg({colname: method}).reset_index().rename(columns={colname: f'{colname}_{method_name}'})
                feats = feats.merge(tmp_df, on='id', how='left')

        # counts
        print("Engineering activity counts data")
        tmp_df = self.activity_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)

        print("Engineering event counts data")
        tmp_df = self.event_counts(df, 'down_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        tmp_df = self.event_counts(df, 'up_event')
        feats = pd.concat([feats, tmp_df], axis=1)

        print("Engineering text change counts data")
        tmp_df = self.text_change_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)

        print("Engineering punctuation counts data")
        tmp_df = self.match_punctuations(df)
        feats = pd.concat([feats, tmp_df], axis=1)

        # input words
        print("Engineering input words data")
        tmp_df = self.get_input_words(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')

        # compare feats
        print("Engineering ratios data")
        feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
        feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
        feats['event_time_ratio'] = feats['event_id_max']  / feats['up_time_max']
        feats['idle_time_ratio'] = feats['action_time_gap1_sum'] / feats['up_time_max']

        print("Done!")
        return feats

In [27]:
preprocessor = Preprocessor(seed=42)

print('Engineering features for training data')
train_feats = preprocessor.make_feats(train_logs)

print()
print('-'*25)
print('Engineering features for test data')
print('-'*25)
test_feats = preprocessor.make_feats(test_logs)

Engineering features for training data
Starting to engineer features
Engineering time data
> for gap 1
> for gap 2
> for gap 3
> for gap 5
> for gap 10
> for gap 20
> for gap 50
> for gap 100
Engineering cursor position data
> for gap 1
> for gap 2
> for gap 3
> for gap 5
> for gap 10
> for gap 20
> for gap 50
> for gap 100
Engineering word count data
> for gap 1
> for gap 2
> for gap 3
> for gap 5
> for gap 10
> for gap 20
> for gap 50
> for gap 100
Engineering statistical summaries for features


100%|██████████| 33/33 [02:01<00:00,  3.68s/it, column=word_count_change100, method=kurtosis]


Engineering activity counts data


100%|██████████| 2471/2471 [00:00<00:00, 7423.94it/s]


Engineering event counts data


...

Engineering text change counts data


100%|██████████| 3/3 [00:00<00:00, 24576.00it/s]


Engineering punctuation counts data


100%|██████████| 3/3 [00:00<00:00, 20695.58it/s]


Engineering input words data
Engineering ratios data
Done!


In [28]:
train_feats.shape

(2471, 230)

In [29]:
train_feats

,id,event_id_max,up_time_max,action_time_sum,action_time_max,action_time_mean,action_time_std,activity_nunique,down_event_nunique,up_event_nunique,text_change_nunique,cursor_position_nunique,cursor_position_max,cursor_position_mean,word_count_nunique,word_count_max,word_count_mean,action_time_gap1_max,action_time_gap1_min,action_time_gap1_mean,action_time_gap1_std,action_time_gap1_sum,action_time_gap1_skew,action_time_gap1_kurtosis,cursor_position_change1_max,cursor_position_change1_mean,cursor_position_change1_std,cursor_position_change1_sum,cursor_position_change1_skew,cursor_position_change1_kurtosis,word_count_change1_max,word_count_change1_mean,word_count_change1_std,word_count_change1_sum,word_count_change1_skew,word_count_change1_kurtosis,action_time_gap2_max,action_time_gap2_min,action_time_gap2_mean,action_time_gap2_std,action_time_gap2_sum,action_time_gap2_skew,action_time_gap2_kurtosis,cursor_position_change2_max,cursor_position_change2_mean,cursor_position_change2_std,cursor_position_change2_sum,cursor_position_change2_skew,cursor_position_change2_kurtosis,word_count_change2_max,word_count_change2_mean,word_count_change2_std,word_count_change2_sum,word_count_change2_skew,word_count_change2_kurtosis,action_time_gap3_max,action_time_gap3_min,action_time_gap3_mean,action_time_gap3_std,action_time_gap3_sum,action_time_gap3_skew,action_time_gap3_kurtosis,cursor_position_change3_max,cursor_position_change3_mean,cursor_position_change3_std,cursor_position_change3_sum,cursor_position_change3_skew,cursor_position_change3_kurtosis,word_count_change3_max,word_count_change3_mean,word_count_change3_std,word_count_change3_sum,word_count_change3_skew,word_count_change3_kurtosis,action_time_gap5_max,action_time_gap5_min,action_time_gap5_mean,action_time_gap5_std,action_time_gap5_sum,action_time_gap5_skew,action_time_gap5_kurtosis,cursor_position_change5_max,cursor_position_change5_mean,cursor_position_change5_std,cursor_position_change5_sum,cursor_position_change5_skew,cursor_position_change5_kurtosis,word_count_change5_max,word_count_change5_mean,word_count_change5_std,word_count_change5_sum,word_count_change5_skew,word_count_change5_kurtosis,action_time_gap10_max,action_time_gap10_min,action_time_gap10_mean,action_time_gap10_std,action_time_gap10_sum,action_time_gap10_skew,action_time_gap10_kurtosis,cursor_position_change10_max,cursor_position_change10_mean,cursor_position_change10_std,cursor_position_change10_sum,cursor_position_change10_skew,cursor_position_change10_kurtosis,word_count_change10_max,word_count_change10_mean,word_count_change10_std,word_count_change10_sum,word_count_change10_skew,word_count_change10_kurtosis,action_time_gap20_max,action_time_gap20_min,action_time_gap20_mean,action_time_gap20_std,action_time_gap20_sum,action_time_gap20_skew,action_time_gap20_kurtosis,cursor_position_change20_max,cursor_position_change20_mean,cursor_position_change20_std,cursor_position_change20_sum,cursor_position_change20_skew,cursor_position_change20_kurtosis,word_count_change20_max,word_count_change20_mean,word_count_change20_std,word_count_change20_sum,word_count_change20_skew,word_count_change20_kurtosis,action_time_gap50_max,action_time_gap50_min,action_time_gap50_mean,action_time_gap50_std,action_time_gap50_sum,action_time_gap50_skew,action_time_gap50_kurtosis,cursor_position_change50_max,cursor_position_change50_mean,cursor_position_change50_std,cursor_position_change50_sum,cursor_position_change50_skew,cursor_position_change50_kurtosis,word_count_change50_max,word_count_change50_mean,word_count_change50_std,word_count_change50_sum,word_count_change50_skew,word_count_change50_kurtosis,action_time_gap100_max,action_time_gap100_min,action_time_gap100_mean,action_time_gap100_std,action_time_gap100_sum,action_time_gap100_skew,action_time_gap100_kurtosis,cursor_position_change100_max,cursor_position_change100_mean,cursor_position_change100_std,cursor_position_change100_sum,cursor_position_change100_skew,cursor_position_change1

In [30]:
nan_cols = train_feats.columns[train_feats.isna().any()].tolist()
nan_cols

['action_time_gap1_skew',
 'action_time_gap1_kurtosis',
 'cursor_position_change1_skew',
 'cursor_position_change1_kurtosis',
 'word_count_change1_skew',
 'word_count_change1_kurtosis',
 'action_time_gap2_skew',
 'action_time_gap2_kurtosis',
 'cursor_position_change2_skew',
 'cursor_position_change2_kurtosis',
 'word_count_change2_skew',
 'word_count_change2_kurtosis',
 'action_time_gap3_skew',
 'action_time_gap3_kurtosis',
 'cursor_position_change3_skew',
 'cursor_position_change3_kurtosis',
 'word_count_change3_skew',
 'word_count_change3_kurtosis',
 'action_time_gap5_skew',
 'action_time_gap5_kurtosis',
 'cursor_position_change5_skew',
 'cursor_position_change5_kurtosis',
 'word_count_change5_skew',
 'word_count_change5_kurtosis',
 'action_time_gap10_skew',
 'action_time_gap10_kurtosis',
 'cursor_position_change10_skew',
 'cursor_position_change10_kurtosis',
 'word_count_change10_skew',
 'word_count_change10_kurtosis',
 'action_time_gap20_skew',
 'action_time_gap20_kurtosis',
 'curs

In [31]:
train_feats = train_feats.drop(columns=nan_cols)
test_feats = test_feats.drop(columns=nan_cols)

train_feats.shape, test_feats.shape

((2471, 182), (3, 182))

In [32]:
train_agg_fe_df = train_logs.groupby('id')[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
train_agg_fe_df.columns = ['_'.join(x) for x in train_agg_fe_df.columns]
train_agg_fe_df = train_agg_fe_df.add_prefix('tmp_')
train_agg_fe_df.reset_index(inplace=True)

In [33]:
test_agg_fe_df = test_logs.groupby('id')[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
test_agg_fe_df.columns = ['_'.join(x) for x in test_agg_fe_df.columns]
test_agg_fe_df = test_agg_fe_df.add_prefix('tmp_')
test_agg_fe_df.reset_index(inplace=True)

In [34]:
train_agg_fe_df.head()

,id,tmp_down_time_mean,tmp_down_time_std,tmp_down_time_min,tmp_down_time_max,tmp_down_time_last,tmp_down_time_first,tmp_down_time_sem,tmp_down_time_median,tmp_down_time_sum,tmp_up_time_mean,tmp_up_time_std,tmp_up_time_min,tmp_up_time_max,tmp_up_time_last,tmp_up_time_first,tmp_up_time_sem,tmp_up_time_median,tmp_up_time_sum,tmp_action_time_mean,tmp_action_time_std,tmp_action_time_min,tmp_action_time_max,tmp_action_time_last,tmp_action_time_first,tmp_action_time_sem,tmp_action_time_median,tmp_action_time_sum,tmp_cursor_position_mean,tmp_cursor_position_std,tmp_cursor_position_min,tmp_cursor_position_max,tmp_cursor_position_last,tmp_cursor_position_first,tmp_cursor_position_sem,tmp_cursor_position_median,tmp_cursor_position_sum,tmp_word_count_mean,tmp_word_count_std,tmp_word_count_min,tmp_word_count_max,tmp_word_count_last,tmp_word_count_first,tmp_word_count_sem,tmp_word_count_median,tmp_word_count_sum
0,001519c8,848180.771998,395112.665961,4526,1801877,1801877,4526,7813.679400,891716.0,2168798234,848297.018772,395105.685655,4557,1801969,1801969,4557,7813.541359,891802.0,2169095477,116.246774,91.797374,0,2259,92,31,1.815369,112.0,297243,711.163473,439.359619,0,1539,1046,0,8.688699,727.0,1818445,128.116152,76.498372,0,256,255,0,1.512819,132.0,327593
1,0022f953,518855.347596,384959.404177,30623,1788842,1788842,30623,7771.013336,407673.0,1273271023,518967.568867,384952.728796,30853,1788969,1788969,30853,7770.878583,407789.0,1273546414,112.221271,55.431189,0,1758,127,230,1.118966,115.0,275391,776.205786,449.133174,0,1676,1519,0,9.066462,743.0,1904809,182.714751,97.763090,0,323,320,0,1.973502,186.0,448382
2,0042269b,828491.775145,489500.796565,4441,1771219,1771219,4441,7611.375322,759582.0,3426641982,828593.612911,489500.438784,4540,1771669,1771669,4540,7611.369759,759674.5,3427063183,101.837766,82.383766,0,3005,450,99,1.281007,94.0,421201,731.611702,592.769763,0,2291,491,0,9.217131,530.5,3025946,194.772727,108.935068,0,404,404,0,1.693860,193.0,805580
3,0059420b,785483.026350,385205.014399,41395,1404394,1404394,41395,9765.334758,848240.5,1222211589,785604.874679,385206.081250,41513,1404469,1404469,41513,9765.361803,848405.5,1222401185,121.848329,113.768226,0,806,75,118,2.884139,110.0,189596,542.537275,307.627200,0,1047,797,0,7.798659,593.5,844188,103.618895,61.882250,0,206,206,0,1.568777,108.5,161231
4,0075873a,713354.197550,405576.409034,78470,1662390,1662390,78470,8061.699636,686588.0,1805499474,713478.141446,405575.631746,78693,1662472,1662472,78693,8061.684185,686728.0,1805813176,123.943896,62.082013,0,701,82,223,1.234013,129.0,313702,600.050968,341.729409,0,1402,1402,0,6.792604,630.0,1518729,125.082971,77.255054,0,252,252,0,1.535610,113.0,316585


In [35]:
test_agg_fe_df.head()

,id,tmp_down_time_mean,tmp_down_time_std,tmp_down_time_min,tmp_down_time_max,tmp_down_time_last,tmp_down_time_first,tmp_down_time_sem,tmp_down_time_median,tmp_down_time_sum,tmp_up_time_mean,tmp_up_time_std,tmp_up_time_min,tmp_up_time_max,tmp_up_time_last,tmp_up_time_first,tmp_up_time_sem,tmp_up_time_median,tmp_up_time_sum,tmp_action_time_mean,tmp_action_time_std,tmp_action_time_min,tmp_action_time_max,tmp_action_time_last,tmp_action_time_first,tmp_action_time_sem,tmp_action_time_median,tmp_action_time_sum,tmp_cursor_position_mean,tmp_cursor_position_std,tmp_cursor_position_min,tmp_cursor_position_max,tmp_cursor_position_last,tmp_cursor_position_first,tmp_cursor_position_sem,tmp_cursor_position_median,tmp_cursor_position_sum,tmp_word_count_mean,tmp_word_count_std,tmp_word_count_min,tmp_word_count_max,tmp_word_count_last,tmp_word_count_first,tmp_word_count_sem,tmp_word_count_median,tmp_word_count_sum
0,0000aaaa,549253.0,298144.503219,338433,760073,760073,338433,210820.0,549253.0,1098506,549339.0,298145.917433,338518,760160,760160,338518,210821.0,549339.0,1098678,86.0,1.414214,85,87,87,85,1.0,86.0,172,0.5,0.707107,0,1,1,0,0.5,0.5,1,0.0,0.000000,0,0,0,0,0.0,0.0,0
1,2222bbbb,501229.0,298012.981358,290502,711956,290502,711956,210727.0,501229.0,1002458,501285.5,298027.830601,290548,712023,290548,712023,210737.5,501285.5,1002571,56.5,14.849242,46,67,46,67,10.5,56.5,113,0.5,0.707107,0,1,1,0,0.5,0.5,1,1.0,0.000000,1,1,1,1,0.0,1.0,2
2,4444cccc,410271.5,318587.667370,184996,635547,184996,635547,225275.5,410271.5,820543,410346.5,318614.537428,185052,635641,185052,635641,225294.5,410346.5,820693,75.0,26.870058,56,94,56,94,19.0,75.0,150,0.5,0.707107,0,1,1,0,0.5,0.5,1,0.5,0.707107,0,1,1,0,0.5,0.5,1


In [36]:
train_feats = train_feats.merge(train_agg_fe_df, on='id', how='left')
test_feats = test_feats.merge(test_agg_fe_df, on='id', how='left')

In [41]:
train_feats.shape, test_feats.shape

((2471, 227), (3, 227))

In [42]:
nan_cols = train_feats.columns[train_feats.isna().any()].tolist()
nan_cols

[]

In [43]:
train_feats = train_feats.merge(train_scores, on='id', how='left')

In [44]:
train_feats.head()

,id,event_id_max,up_time_max,action_time_sum,action_time_max,action_time_mean,action_time_std,activity_nunique,down_event_nunique,up_event_nunique,text_change_nunique,cursor_position_nunique,cursor_position_max,cursor_position_mean,word_count_nunique,word_count_max,word_count_mean,action_time_gap1_max,action_time_gap1_min,action_time_gap1_mean,action_time_gap1_std,action_time_gap1_sum,cursor_position_change1_max,cursor_position_change1_mean,cursor_position_change1_std,cursor_position_change1_sum,word_count_change1_max,word_count_change1_mean,word_count_change1_std,word_count_change1_sum,action_time_gap2_max,action_time_gap2_min,action_time_gap2_mean,action_time_gap2_std,action_time_gap2_sum,cursor_position_change2_max,cursor_position_change2_mean,cursor_position_change2_std,cursor_position_change2_sum,word_count_change2_max,word_count_change2_mean,word_count_change2_std,word_count_change2_sum,action_time_gap3_max,action_time_gap3_min,action_time_gap3_mean,action_time_gap3_std,action_time_gap3_sum,cursor_position_change3_max,cursor_position_change3_mean,cursor_position_change3_std,cursor_position_change3_sum,word_count_change3_max,word_count_change3_mean,word_count_change3_std,word_count_change3_sum,action_time_gap5_max,action_time_gap5_min,action_time_gap5_mean,action_time_gap5_std,action_time_gap5_sum,cursor_position_change5_max,cursor_position_change5_mean,cursor_position_change5_std,cursor_position_change5_sum,word_count_change5_max,word_count_change5_mean,word_count_change5_std,word_count_change5_sum,action_time_gap10_max,action_time_gap10_min,action_time_gap10_mean,action_time_gap10_std,action_time_gap10_sum,cursor_position_change10_max,cursor_position_change10_mean,cursor_position_change10_std,cursor_position_change10_sum,word_count_change10_max,word_count_change10_mean,word_count_change10_std,word_count_change10_sum,action_time_gap20_max,action_time_gap20_min,action_time_gap20_mean,action_time_gap20_std,action_time_gap20_sum,cursor_position_change20_max,cursor_position_change20_mean,cursor_position_change20_std,cursor_position_change20_sum,word_count_change20_max,word_count_change20_mean,word_count_change20_std,word_count_change20_sum,action_time_gap50_max,action_time_gap50_min,action_time_gap50_mean,action_time_gap50_std,action_time_gap50_sum,cursor_position_change50_max,cursor_position_change50_mean,cursor_position_change50_std,cursor_position_change50_sum,word_count_change50_max,word_count_change50_mean,word_count_change50_std,word_count_change50_sum,action_time_gap100_max,action_time_gap100_min,action_time_gap100_mean,action_time_gap100_std,action_time_gap100_sum,cursor_position_change100_max,cursor_position_change100_mean,cursor_position_change100_std,cursor_position_change100_sum,word_count_change100_max,word_count_change100_mean,word_count_change100_std,word_count_change100_sum,activity_0_count,activity_1_count,activity_2_count,activity_3_count,activity_4_count,down_event_0_count,down_event_1_count,down_event_2_count,down_event_3_count,down_event_4_count,down_event_5_count,down_event_6_count,down_event_7_count,down_event_8_count,down_event_9_count,down_event_10_count,down_event_11_count,down_event_12_count,down_event_13_count,down_event_14_count,down_event_15_count,up_event_0_count,up_event_1_count,up_event_2_count,up_event_3_count,up_event_4_count,up_event_5_count,up_event_6_count,up_event_7_count,up_event_8_count,up_event_9_count,up_event_10_count,up_event_11_count,up_event_12_count,up_event_13_count,up_event_14_count,up_event_15_count,text_change_0_count,text_change_1_count,text_change_2_count,text_change_3_count,text_change_4_count,text_change_5_count,text_change_6_count,text_change_7_count,text_change_8_count,text_change_9_count,text_change_10_count,text_change_11_count,text_change_12_count,text_change_13_count,text_change_14_count,punct_cnt,input_word_count,input_word_length_mean,input_word_length_max,input_word_length_std,word_time_ratio,word_event_ratio,event_time_ratio,idle_time_ratio,tmp_down_time_mean,

In [45]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

train_feats['score_class'] = le.fit_transform(train_feats['score'])

## (5) Base Model